# Assigment 4

# This is a mini-project assignment that includes only programming questions. You are asked to implement optimization algorithms for ML classification problems. 

## Marking of this assignment will be based on the correctness of your ML pipeline and efficiency of your code. 

## Upload your code on Learn dropbox and submit pdfs of the code and to Crowdmark.

## -----------------------------------------------------------------------------------------------------------

In [1]:
# !pip install numpy, scipy, sys

## Suggested way of loading data to python for the assigment. There are alternatives of course, you can use your preferred way if you want.

In [2]:
# Download the LIBSVM package from here: https://www.csie.ntu.edu.tw/~cjlin/libsvm/#download 
# If your download is successfull you should have the folder with name: libsvm-3.24.
# We will use this package to load datasets. 

# Enter the downloaded folder libsvm-3.24 through your terminal. 
# Run make command to compile the package.

# Load this auxiliary package.
import sys
import os
# add here your path to the folder libsvm-3.24/python
path = os.getcwd()+'/libsvm-3.24/python/'

print(os.getcwd())
# Add the path to the Python paths so Python can find the module.
sys.path.append(path)
# sys.path.append(os.getcwd()+'/libsvm-3.24/')
# sys.path.append(os.getcwd()+'/libsvm-3.24/python/')
print(path)


# Load the LIBSVM module.
from svmutil import *

# Add here your path to the folder libsvm-3.24
path = './libsvm-3.24/heart_scale'

# Test that it works. This will load the data "heart_scale" 
# and it will store the labels in "b" and the data matrix in "A".
b, A = svm_read_problem(path)

# Use "svm_read_problem" function to load data for your assignment.

# Note that matrix "A" stores the data in a sparse format. 
# In particular matrix "A" is a list of dictionaries. 
# The length of the list gives you the number of samples.
# Each entry in the list is a dictionary. The keys of the dictionary are the non-zero features.
# The values of the dictionary for each key is a list which gives you the feature value.


/home/ned/Desktop/CS794/A4
/home/ned/Desktop/CS794/A4/libsvm-3.24/python/


## Load other useful modules

In [3]:
# Numpy is useful for handling arrays and matrices.
import numpy as np


import matplotlib.pyplot as plt


# my import 
from numpy.linalg import norm
import math
import time
from scipy import real, ndimage
from scipy.sparse import *



## Datasets that you will need for this assignment.

In [4]:
# There is an extended selection of classification and regression datasets 
# https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/

# Out of all these datasets you will need the following 3 datasets, which are datasets for classification problems.
# 
# a9a dataset: https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#a9a 
# This dataset is small, it is recommened to start your experiments with this dataset.
#
# epsilon dataset: https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#epsilon 
# This dataset relatively large, but most importantly it is ill-conditioned. This means that the
# methods might stagnate fast.
#
# kdd2010 (algebra): https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#kdd2010%20(algebra)
# This dataset is large and very sparse. Exploit the sparsity of the problem when you implement optimization methods.

## Training, Validation and Testing data

In [5]:
# All datasets above consist of training and testing data. 

# You should seperate the training data into training and validation data.
# Follow the instructions from the lectures about how you can use both training and validation data.
# You can use 10% of the training data as validation data and the remaining 90% to train the models.
# This is a suggested percentage, you can do otherwise if you wish.

# Do not use the testing data to influence training in any way. Do not use the testing data at all.
# Only your instructor and TA will use the testing data to measure generalization error. 
# If you do use the testing data to tune parameters or for training of the algorithms we will figure it out :-). 

## Optimization problems

### You need to solve the following optimization problems 

Hinge-loss
$$\mbox{minimize}_{x\in\mathbb{R}^d, \beta \in \mathbb{R}} \ \frac{1}{n} \sum_{i=1}^n \max \{0,1-b_i(a_i^Tx + \beta)\},$$
where $a_i\in\mathbb{R}^d$ is the feature vector for sample $i$ and $b_i$ is the label of sample $i$. The sub-gradient of the hinge-loss is given in the lecture slides (note that there is a small difference due to the intercept $\beta$). A smooth approximation of the function $f(z):=\max\{0,1-z\}$ is given by
$$
\psi_\mu(z) = 
\begin{cases}
0 & z\ge 1\\
1-z^2 & \mu < z < 1 \\
(1-\mu)^2 + 2(1-\mu)(\mu-z) & z \le \mu.
\end{cases}
$$
You can use the smooth approximation $\psi_\mu(z)$ for methods that work only for smooth functions. For sub-gradient methods you should use the sub-gradient.

L2-regularized logistic regression
$$\mbox{minimize}_{x\in\mathbb{R}^d,\beta\in\mathbb{R}} \ \lambda \|x\|_2^2 + \frac{1}{n} \sum_{i=1}^n \log (1+ \exp(-b_i(a_i^Tx + \beta))).$$
This is a smooth objective function, therefore, you should use gradient methods to solve it. You do not need sub-gradient methods for this problem.

Note that for hinge-loss the labels should be -1 or 1. For logistic regression the labels should be 0 or 1. Check the labels that you loaded to make sure that they have the correct values for the correct problem. If not, then convert them appropriately.

## Optimization algorithms

In [6]:
# For this assignment you will need the following methods

# 1) Stochastic sub-gradient
# 2) Stochastic gradient
# 3) Mini-batch (sub-)gradient (you will have to decide what batching strategy to use, see lecture slides)
# 4) Stochastic average sub-gradient (SAG)
# 5) Stochastic average gradient (SAG)
# 6) Gradient descent with Armijo line-search
# 7) Acceleratd gradient with Armijo line-search (the same method as Q5 in Assignemnt 3)

# Information is provided in the lecture slides about parameter tuning and termination.
# However, the final decision of any parameter tuning and termination criteria is up to the students to make. 

## Validation error: measure the validation error by calculating
$$
\frac{1}{t}\sum_{i\in\mbox{validation data}} \left| \ b_i^{\mbox{your model}} - b_i^{\mbox{true}} \ \right|
$$
where $t$ is the number of samples in your validation set. $b_i^{\mbox{true}}$ is the true label of the $i$-th sample. $b_i^{\mbox{your model}}$ is the label of the $i$-th sample of your model.

For hinge loss calculate $$b_i^{\mbox{your model}}:= \mbox{sign}(a_i^Tx + \beta).$$

For logistic regression calculate the predicted label by
$$
b_i^{\mbox{your model}}=
\begin{cases}
1 & \mbox{if } a_i^Tx + \beta > 0\\
0 & \mbox{otherwise}
\end{cases}
$$

## Question 1: Use the ML pipeline that is mentioned in slide 60 of Lecture 11 to train your model for the logistic regression problem (the hinge-loss problem does not have any hyper-parameters). Pick any algorithm that you want from the above suggested list to train the models. Report your ML pipeline. Print your Generalization Error. We will not measure running time for this pipeline. Running time will be measure only in Q2.

## Question 2: Plot the objective function (y-axis) vs running time in sec (x-axis). Have one plot for each optimization problem. In each plot show the performance of all relevant algorithms. For each plot use the parameter setting that gives you the best validation error in Q1 (this refers to the logistic regression probelm). Do not show plots for all parameter settings that you tried in Q1, only for the one that gives you the smallest validation error. Do not include computation of any plot data in the computation of the running time of the algorithm, unless the plot data are computed by the algorithm anyway. Make sure that the plots are clean and use appropriate legends. Note that we should be able to re-run the code and obtain the plots. 

### For this question, we will measure the running time of your stochastic sub-gradient method for the sparse dataset kdd2010 (algebra) for the hinge-loss problem. We will not measure the running time of any other combination of algorithm, dataset, problem. You need to implement the stochastic sub-gradient method and encapsulate it in a python class.

To make sure your object can be used by our script, your class should have two methods:

1. <strong>fit(self, train_data, train_label)</strong>. It will use stochastic sub-gradient method to minimize the hinge loss and store the optimized coefficients (i.e. $x, \beta$) in the instance. The "train_data" and "train_label" are similar to the output of "svm_read_problem". 
    * "train_data" is a list of $n$ python dictionaries (int -> float), which presents a sparse matrix. The keys (int) and values (float) in the dictionary at train_data[i] are the indices (int) and values (float) of non-zero entries of row $i$. 
    * "train_label" is a list of $n$ integers, it only has <strong>-1s and 1s</strong>. $n$ is the number of samples.  This function returns nothing.


2. <strong>predict(self, test_data)</strong>. It will predict the label of the input "test_data" by using the coefficients stored in the instance. The "test_data" has the same data structure as the "train_data" of the "fit" function. This function returns a list of <strong>-1s and 1s</strong> (i.e. the prediction of your labels).

You can also define other methods to help your programming, we will only call the two methods decribed above.

To let us import your class, you need to follow these rules:

1. You should name your python file by <strong>a4_[your student ID].py</strong>. For example, if your student id is 12345, then your file name is <strong>a4_12345.py</strong>
1. Your object name should be <strong>MyMethod</strong> (it's case sensitive).

Any violation of the above requirements will get error in our script and you will get at most 50% of the total score. Your solution will be mainly measured by the runing time of the <strong>fit</strong> function and the accuracy of the <strong>predict</strong> function. For example your method will be called and measured in following pattern:

    obj = MyMethod()
    st = time.time()
    obj.fit(train_data, train_label) # .fit() optimizes the objective and stores coefficients in obj.
    running_time = time.time() - st
    predict_label = obj.predict(test_data)
    accuracy = get_accuracy(predict_label, test_label) # this is a function we use to measure accuracy.
Then your accuracy will be measured by <strong>predict_labels</strong>, you don't have to implement "get_accuracy". When you finish your implementation, upload the .py file to Learn dropbox.

In [7]:
########################################################################
# Q1
########################################################################

# Read from a9a
from scipy.sparse import *
from sklearn.feature_extraction import DictVectorizer

# A is a list of Dictionary, b is a list of int. 
b,A = svm_read_problem(os.getcwd()+'/a9a')
# 90% training and 10%testing
A_training = A[:int(0.9*len(A))]
b_training = b[:int(0.9*len(b))]
A_testing = A[int(0.9*len(A)):]
b_testing = b[int(0.9*len(b)):]


# of columns
features = 123
print(features)

# A_matrix is scipy.sparse.csr_matrix, 
vec = DictVectorizer()
A_matrix = vec.fit_transform(A_training)

b_column = np.array(b_training).reshape(len(b_training),1)
x = csr_matrix(np.ones(shape=(features,1), dtype=float), shape=(features,1))



123


In [17]:

# help functions
# A_matrix is scipy.sparse.csr_matrix, x is scipy.sparse.csr.csr_matrix, b is numpy.ndarray 
import random,math
def get_hinge_loss(A,x,b):
#     print('get_hinge_loss')
    A_dot_x = A.dot(x)
    ei = A_dot_x.multiply(b).tocsr()  # scipy.sparse.coo.coo_matrix ->csr a matrix 
    one_error = lambda ei : max(0,1-ei)
    total_error = sum(map(one_error,[ei[i,0] for i in range(ei.shape[0])]))
    return total_error/A.shape[0]
    
# return a csr_matrix (row vector)
def get_gradient(A,x,b,i):
#     print('get_gradient')
    if 1 - (b[i]*(A[i].dot(x))[0,0]) > 0:
        grad = (-b[i]*A[i]).reshape(features,1)
#         print(type(grad),grad.shape)
        return grad
    else: 
        grad = np.zeros(shape=(features,1), dtype=float)
#         print(type(grad),grad.shape)
        return grad
        
        
def stochastic_subgradient(A,x,b,max_iteration,epsilon):
    print(x.shape)
    count = 2
    print('stochastic_subgradient')
    hinge_loss = epsilon # initial a size to start 
    while count < max_iteration and hinge_loss >= epsilon:
        index = random.randint(0,len(b))
        hinge_loss = get_hinge_loss(A,x,b)
        grad = get_gradient(A,x,b,index)
        step_size = 1/math.log(count,2)   
#         print('count %d\t i %d\t stepsize %.4f \t hinge_loss %.4f\t sum_of_grad %d'%(count,index, step_size,hinge_loss, sum(grad)))
        # stepsize = 1/couont
  
        
        x = x - step_size*csr_matrix(grad,shape=(grad.shape[0],1))
        count +=1
    return (x,count,index,hinge_loss)
solution = stochastic_subgradient(A_matrix,x,b_column, 1000,0.5)
print(solution)
print(solution[3])



# def get_hinge_loss(a,x,b):
#     term = 1-b*a.dot(x)
#     term = np.where(term>0, term , 0)
#     result = (1/b.shape[0])*np.sum(term)
#     return result

# def get_gradient(x,bi,ai):
#     term = bi*ai.dot(x)
#     if 1-term>0:
#         result = -bi*ai
#         result = np.reshape(result,(len(result),1))
#     else:
#         result = np.zeros((x.shape[0],1))
#     return result

# def stochastic_subgradient(A,x,b,max_iteration,epsilon):
#     print(x.shape)
#     count = 0
#     print('stochastic_subgradient')
#     hinge_loss = epsilon # initial a size to start 
#     while count < max_iteration and hinge_loss >= epsilon:
#         index = random.randint(0,len(b))
#         hinge_loss = get_hinge_loss(A,x,b)
#         grad = get_gradient(A,x,b,index)
#         print('count %d\t random_i %d\t hinge_loss %.4f\t sum_of_grad %d'%(count,index,hinge_loss, sum(grad)))
#         # stepsize = 1/couont
#         step_size = 1/(count+1)        
#         if sum(grad) != 0:
#             x = x - step_size*csr_matrix(grad,shape=(grad.shape[0],1))
#             count +=1
        
# stochastic_subgradient(A_matrix,x,b_column, 2000,0.01)

(123, 1)
stochastic_subgradient
(<123x1 sparse matrix of type '<class 'numpy.float64'>'
	with 123 stored elements in Compressed Sparse Row format>, 407, 13359, 0.49527455732598585)
0.49527455732598585


In [ ]:
# Question2:
class MyMethod(object):
